<a href="https://colab.research.google.com/github/SrijaG29/Email_spam_classification/blob/main/Email_spam_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'email-classification-ham-spam:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4812840%2F8140416%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240911%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240911T055109Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D21a321fca2f87017b8301e242c544d2d1f05cdcdce14fabd11f1af1fadf31a586d80fca5dead3c9d108bc82d5a1dd9708e895dfa6f754498cc00b41d2234ed9b83afd0d2464e2a1d64d23479baf540d645c50743ddf4b17285d7f36e46b979d3c30cc46f1f18f5eab3c48be18e3f140bccba4f3f6125170e8b8f29ff6695182b4057e324522307a5dfcd321aac6ce80122018b094ba6420a3dc773f735bb2c8cf551102488d3d909e096034f249df3f04cc530963bfb04a604b898f2876118d12927408caf5861902ef2f66bf7bc0fa408c175951bc7db96840467d044e971f2fa13b6ebde05a4e6e137f2bcea26efc80cc61688cfb893c30ab6c1ffc6dffe37'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 4287 bytes downloaded
Downloaded and uncompressed: email-classification-ham-spam
Data source import complete.


In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/email-classification-ham-spam/email_classification.csv


In [4]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=becd123374ce885f9b953282d977dba56050b00c6313e6637225037c102bebe1
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [5]:
from pyspark.sql import SparkSession

spark = (
    SparkSession
    .builder
    .appName("Spam Classification")
    .getOrCreate()
)

In [6]:
email_df = spark.read.format('csv').option('header',True).load('/kaggle/input/email-classification-ham-spam/email_classification.csv')
email_df.show(truncate = False)

+-------------------------------------------------------------------------------------------------+-----+
|email                                                                                            |label|
+-------------------------------------------------------------------------------------------------+-----+
|Upgrade to our premium plan for exclusive access to premium content and features.                |ham  |
|Happy holidays from our team! Wishing you joy and prosperity this season.                        |ham  |
|We're hiring! Check out our career opportunities and join our dynamic team.                      |ham  |
|Your Amazon account has been locked. Click here to verify your account information.              |spam |
|Your opinion matters! Take our survey and help us enhance your experience.                       |ham  |
|Your payment has been received. Thank you for your prompt transaction.                           |ham  |
|Your email account storage is full. Click her

In [7]:
email_df_pd = email_df.toPandas()
email_df_pd

,email,label
0,Upgrade to our premium plan for exclusive acce...,ham
1,Happy holidays from our team! Wishing you joy ...,ham
2,We're hiring! Check out our career opportuniti...,ham
3,Your Amazon account has been locked. Click her...,spam
4,Your opinion matters! Take our survey and help...,ham
...,...,...
174,We're pleased to inform you that your refund h...,ham
175,Get rich quick! Invest in our revolutionary ne...,spam
176,Your free trial period is ending soon. Upgrade...,ham
177,Your order is on its way! Track your shipment ...,ham


In [8]:
email_df_pd.isna().sum()

,0
email,0
label,0


In [9]:
from pyspark.sql.functions import count,col
email_df.groupBy('label').agg(count(col('email'))).show()

+-----+------------+
|label|count(email)|
+-----+------------+
|  ham|         100|
| spam|          79|
+-----+------------+



In [10]:
email_df_copy = email_df.select('*')
email_df_copy.show()

+--------------------+-----+
|               email|label|
+--------------------+-----+
|Upgrade to our pr...|  ham|
|Happy holidays fr...|  ham|
|We're hiring! Che...|  ham|
|Your Amazon accou...| spam|
|Your opinion matt...|  ham|
|Your payment has ...|  ham|
|Your email accoun...| spam|
|Dear [Name], than...|  ham|
|Your account has ...|  ham|
|You've been chose...| spam|
|Don't miss out on...|  ham|
|We're hiring inte...|  ham|
|You're pre-approv...| spam|
|We're thrilled to...|  ham|
|We're excited to ...|  ham|
|We've added new f...|  ham|
|You're a winner! ...| spam|
|Your Facebook acc...| spam|
|Congratulations o...|  ham|
|Congratulations! ...| spam|
+--------------------+-----+
only showing top 20 rows



In [11]:
from pyspark.sql.functions import when,col

email_df_copy = email_df_copy.withColumn('label',when(col('label')=='ham', 0).when(col('label')=='spam',1))
email_df_copy.show()

+--------------------+-----+
|               email|label|
+--------------------+-----+
|Upgrade to our pr...|    0|
|Happy holidays fr...|    0|
|We're hiring! Che...|    0|
|Your Amazon accou...|    1|
|Your opinion matt...|    0|
|Your payment has ...|    0|
|Your email accoun...|    1|
|Dear [Name], than...|    0|
|Your account has ...|    0|
|You've been chose...|    1|
|Don't miss out on...|    0|
|We're hiring inte...|    0|
|You're pre-approv...|    1|
|We're thrilled to...|    0|
|We're excited to ...|    0|
|We've added new f...|    0|
|You're a winner! ...|    1|
|Your Facebook acc...|    1|
|Congratulations o...|    0|
|Congratulations! ...|    1|
+--------------------+-----+
only showing top 20 rows



In [12]:
email_df_copy.printSchema()

root
 |-- email: string (nullable = true)
 |-- label: integer (nullable = true)



In [13]:
from pyspark.ml.feature import StringIndexer

label_indexer = StringIndexer(inputCol = 'label',outputCol= 'label_Encoded')

label_model = label_indexer.fit(email_df_copy)
email_df_copy = label_model.transform(email_df_copy)

In [14]:
email_indexer = StringIndexer(inputCol = 'email',outputCol= 'email_Encoded')

email_model = email_indexer.fit(email_df_copy)
email_df_copy = email_model.transform(email_df_copy)

In [15]:
email_df_copy.show()

+--------------------+-----+-------------+-------------+
|               email|label|label_Encoded|email_Encoded|
+--------------------+-----+-------------+-------------+
|Upgrade to our pr...|    0|          0.0|         74.0|
|Happy holidays fr...|    0|          0.0|         51.0|
|We're hiring! Che...|    0|          0.0|         97.0|
|Your Amazon accou...|    1|          1.0|        123.0|
|Your opinion matt...|    0|          0.0|        141.0|
|Your payment has ...|    0|          0.0|        146.0|
|Your email accoun...|    1|          1.0|         25.0|
|Dear [Name], than...|    0|          0.0|         37.0|
|Your account has ...|    0|          0.0|        130.0|
|You've been chose...|    1|          1.0|         13.0|
|Don't miss out on...|    0|          0.0|         44.0|
|We're hiring inte...|    0|          0.0|         96.0|
|You're pre-approv...|    1|          1.0|        119.0|
|We're thrilled to...|    0|          0.0|        109.0|
|We're excited to ...|    0|   

In [16]:
model_input = email_df_copy.drop('email','label')
model_input.show()

+-------------+-------------+
|label_Encoded|email_Encoded|
+-------------+-------------+
|          0.0|         74.0|
|          0.0|         51.0|
|          0.0|         97.0|
|          1.0|        123.0|
|          0.0|        141.0|
|          0.0|        146.0|
|          1.0|         25.0|
|          0.0|         37.0|
|          0.0|        130.0|
|          1.0|         13.0|
|          0.0|         44.0|
|          0.0|         96.0|
|          1.0|        119.0|
|          0.0|        109.0|
|          0.0|         86.0|
|          0.0|        110.0|
|          1.0|        117.0|
|          1.0|         17.0|
|          0.0|         29.0|
|          1.0|         34.0|
+-------------+-------------+
only showing top 20 rows



In [17]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols = ['email_Encoded'],outputCol = 'Email_Details')
model_input = assembler.transform(model_input)

model_input.show()

+-------------+-------------+-------------+
|label_Encoded|email_Encoded|Email_Details|
+-------------+-------------+-------------+
|          0.0|         74.0|       [74.0]|
|          0.0|         51.0|       [51.0]|
|          0.0|         97.0|       [97.0]|
|          1.0|        123.0|      [123.0]|
|          0.0|        141.0|      [141.0]|
|          0.0|        146.0|      [146.0]|
|          1.0|         25.0|       [25.0]|
|          0.0|         37.0|       [37.0]|
|          0.0|        130.0|      [130.0]|
|          1.0|         13.0|       [13.0]|
|          0.0|         44.0|       [44.0]|
|          0.0|         96.0|       [96.0]|
|          1.0|        119.0|      [119.0]|
|          0.0|        109.0|      [109.0]|
|          0.0|         86.0|       [86.0]|
|          0.0|        110.0|      [110.0]|
|          1.0|        117.0|      [117.0]|
|          1.0|         17.0|       [17.0]|
|          0.0|         29.0|       [29.0]|
|          1.0|         34.0|   

In [18]:
model_input = model_input.drop('email_Encoded')
model_input.show()

+-------------+-------------+
|label_Encoded|Email_Details|
+-------------+-------------+
|          0.0|       [74.0]|
|          0.0|       [51.0]|
|          0.0|       [97.0]|
|          1.0|      [123.0]|
|          0.0|      [141.0]|
|          0.0|      [146.0]|
|          1.0|       [25.0]|
|          0.0|       [37.0]|
|          0.0|      [130.0]|
|          1.0|       [13.0]|
|          0.0|       [44.0]|
|          0.0|       [96.0]|
|          1.0|      [119.0]|
|          0.0|      [109.0]|
|          0.0|       [86.0]|
|          0.0|      [110.0]|
|          1.0|      [117.0]|
|          1.0|       [17.0]|
|          0.0|       [29.0]|
|          1.0|       [34.0]|
+-------------+-------------+
only showing top 20 rows



In [19]:
train_data,test_data = model_input.randomSplit([0.75,0.25],seed = 42)

In [20]:
train_data.show()

+-------------+-------------+
|label_Encoded|Email_Details|
+-------------+-------------+
|          0.0|       [29.0]|
|          0.0|       [30.0]|
|          0.0|       [32.0]|
|          0.0|       [35.0]|
|          0.0|       [37.0]|
|          0.0|       [39.0]|
|          0.0|       [42.0]|
|          0.0|       [43.0]|
|          0.0|       [44.0]|
|          0.0|       [47.0]|
|          0.0|       [48.0]|
|          0.0|       [50.0]|
|          0.0|       [51.0]|
|          0.0|       [52.0]|
|          0.0|       [54.0]|
|          0.0|       [57.0]|
|          0.0|       [58.0]|
|          0.0|       [60.0]|
|          0.0|       [61.0]|
|          0.0|       [62.0]|
+-------------+-------------+
only showing top 20 rows



In [21]:
train_data.count()

140

In [22]:
test_data.show()

+-------------+-------------+
|label_Encoded|Email_Details|
+-------------+-------------+
|          0.0|       [31.0]|
|          0.0|       [38.0]|
|          0.0|       [40.0]|
|          0.0|       [41.0]|
|          0.0|       [46.0]|
|          0.0|       [53.0]|
|          0.0|       [59.0]|
|          0.0|       [65.0]|
|          0.0|       [66.0]|
|          0.0|       [68.0]|
|          0.0|       [70.0]|
|          0.0|       [71.0]|
|          0.0|       [79.0]|
|          0.0|       [82.0]|
|          0.0|       [83.0]|
|          0.0|       [84.0]|
|          0.0|       [86.0]|
|          0.0|       [88.0]|
|          0.0|       [92.0]|
|          0.0|       [99.0]|
+-------------+-------------+
only showing top 20 rows



In [23]:
test_data.count()

39

In [24]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(featuresCol='Email_Details', labelCol='label_Encoded')
lr_model = lr.fit(train_data)
lr_predictions = lr_model.transform(test_data)


In [65]:
lr_predictions.show()

+-------------+-------------+--------------------+--------------------+----------+
|label_Encoded|Email_Details|       rawPrediction|         probability|prediction|
+-------------+-------------+--------------------+--------------------+----------+
|          0.0|       [31.0]|[-0.8104176320022...|[0.30780150799094...|       1.0|
|          0.0|       [38.0]|[-0.5901966285324...|[0.35658973986172...|       1.0|
|          0.0|       [40.0]|[-0.5272763418267...|[0.37115236177818...|       1.0|
|          0.0|       [41.0]|[-0.4958161984739...|[0.37852438025313...|       1.0|
|          0.0|       [46.0]|[-0.3385154817097...|[0.41617012931490...|       1.0|
|          0.0|       [53.0]|[-0.1182944782399...|[0.47046081899202...|       1.0|
|          0.0|       [59.0]|[0.07046638187700...|[0.51760930947058...|       0.0|
|          0.0|       [65.0]|[0.25922724199398...|[0.56444632128949...|       0.0|
|          0.0|       [66.0]|[0.29068738534681...|[0.57216440791094...|       0.0|
|   

In [26]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(labelCol='label_Encoded', predictionCol="prediction", metricName="accuracy")
accuracy_lr = evaluator.evaluate(lr_predictions)
print(f"Accuracy for Logistic Classification is: {accuracy_lr}")

Accuracy for Logistic Classification is: 0.7948717948717948


In [66]:
from pyspark.sql.types import StructField,StructType,StringType

user_schema = StructType([StructField('email',StringType(),True)])

In [67]:
details = [['Your Amazon account has been locked. Click here to verify your account information']]

In [68]:
input_details = spark.createDataFrame(details, schema=user_schema)
input_details.show()

+--------------------+
|               email|
+--------------------+
|Your Amazon accou...|
+--------------------+



In [69]:
input_details = email_model.transform(input_details)

In [57]:
input_details.printSchema()

root
 |-- email: string (nullable = true)
 |-- email_Encoded: double (nullable = false)



In [ ]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols = ['email_Encoded'],outputCol = 'Email_Details')
input_details = assembler.transform(input_details)

In [60]:
input_details.printSchema()

root
 |-- email: string (nullable = true)
 |-- email_Encoded: double (nullable = false)
 |-- Email_Details: vector (nullable = true)



In [62]:
input_to_model = input_details.select('Email_Details')
input_to_model.printSchema()

root
 |-- Email_Details: vector (nullable = true)



In [ ]:
prediction1 = model_rf.transform(input_to_model)

In [63]:
lr1_predictions = lr_model.transform(input_to_model)

In [64]:
lr1_predictions.show()

Py4JJavaError: An error occurred while calling o584.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 59.0 failed 1 times, most recent failure: Lost task 0.0 in stage 59.0 (TID 54) (3c188846e8d1 executor driver): org.apache.spark.SparkException: [FAILED_EXECUTE_UDF] Failed to execute user defined function (`StringIndexerModel$$Lambda$3863/0x00000008415fa040`: (string) => double).
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala:198)
	at org.apache.spark.sql.errors.QueryExecutionErrors.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenEvaluatorFactory$WholeStageCodegenPartitionEvaluator$$anon$1